In [1]:
''' 
    Impor Pustaka apa yang kita butuhkan
'''
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
import time

In [2]:
''' 
    Config Mediapipe Solution
'''
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [3]:
''' 
    Pojok Dir => Tempat untuk memeriksa metode/atribut
'''
print(dir(mp_face_mesh))
# print(dir(cv2))

['FACEMESH_CONTOURS', 'FACEMESH_FACE_OVAL', 'FACEMESH_IRISES', 'FACEMESH_LEFT_EYE', 'FACEMESH_LEFT_EYEBROW', 'FACEMESH_LEFT_IRIS', 'FACEMESH_LIPS', 'FACEMESH_NOSE', 'FACEMESH_NUM_LANDMARKS', 'FACEMESH_NUM_LANDMARKS_WITH_IRISES', 'FACEMESH_RIGHT_EYE', 'FACEMESH_RIGHT_EYEBROW', 'FACEMESH_RIGHT_IRIS', 'FACEMESH_TESSELATION', 'FaceMesh', 'NamedTuple', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'association_calculator_pb2', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inference_calculator_pb2', 'landmarks_refinement_calculator_pb2', 'logic_calculator_pb2', 'non_max_suppression_calculator_pb2', 'np', 'rect_transformation_calculator_pb2', 'split_vector_calculator_pb2', 'ssd_anchors_calculator_pb2', 'tensors_to_classification_calculator_pb2', 'tensors_to_detections_calculator_pb2', 'tensors_to_landmarks_c

'''
    Define Landmark yang kita butuhkan
'''
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [ 38, 76, 268, 292, 86, 316, ]
nose_idxs = [1]
forehead_idxs = [8]


nose_tip = [1]
right_eye = [33]
left_eye = [263]
lips_upper = [61, 291]
chin = [199]



In [4]:
def diffTime(looking_down_timer):
    diff = (datetime.now() - looking_down_timer).total_seconds()
    return diff 


def putText(frame, val, x, y, unity=""):
    if len(unity) == 0:
        cv2.putText(frame, f'{val}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, f'{val:.4f} {unity}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
def getOurLandmark():
    our_landmarks = []
    for idxs in (chosen_left_eye_idxs, chosen_right_eye_idxs, chosen_mouth_idxs, nose_idxs, forehead_idxs):
        our_landmarks.extend(idxs)
    return our_landmarks
        
def getTargettedLandmark():
    targetted_landmark = []
    targetted_landmark.extend(item for sublist in [nose_tip, right_eye, left_eye, lips_upper, chin] for item in sublist)  
    return targetted_landmark    

def headPose(image, face_landmarks, face_2d, face_3d, nose_2d, nose_3d, required_looking_time):
    # Mendapatkan dimensi gambar
    img_h, img_w, img_c = image.shape
    targetted_landmark = getTargettedLandmark()

    # Konversi koordinat menjadi array NumPy
    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)
    
    # Matriks kamera
    focal_length = 1 * img_w
    cam_matrix = np.array([
                            [focal_length, 0, img_h / 2],
                            [0, focal_length, img_w / 2],
                            [0, 0, 1]
                        ])
    dist_matrix = np.zeros((4, 1), dtype=np.float64)
    
    # Hitung orientasi kepala menggunakan solvePnP
    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    # Matriks Rotasi
    rmat, jac = cv2.Rodrigues(rot_vec)
    # Angle
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
    
    # Mendapatkan Derajat Rotasi Y
    x = angles[0] * 360
    y = angles[1] * 360
    z = angles[2] * 360
    
    # Melihat kemiringan kepala pengguna
    if y < -10:
        text = 'Looking Left'
    elif y > 10:
        text = 'Looking Right'
    elif x < -7:
        text = 'Looking Down'
    elif x > 10:
        text = 'Looking Up'
    else:
        text = 'Forward'
        
    global looking_down_timer
    
    if text == 'Looking Down' and looking_down_timer == None:
        looking_down_timer = datetime.now()
    elif text != 'Looking Down': 
        looking_down_timer = None
        
    if looking_down_timer is not None :
        diff_time = diffTime(looking_down_timer)
        putText(image, diff_time, 20, 100, 'seconds')
    
        if diff_time > required_looking_time:
            putText(image, "GO SLEEP NOW", 20, 150)
        print(looking_down_timer, '\t', diff_time)
        
    # Memproyeksikan koordinat hidung ke 3d
    nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
    
    # Gambar Arah Pandangan
    p1 = (int(nose_2d[0]), int(nose_2d[1]))
    p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
    cv2.line(image, p1, p2, (255, 0, 0), 3)
    
    return p1, p2, text, x, y, z 
    
    
    
def praHeadPose(img, idx, lm):
    img_h, img_w, img_c = image.shape
    global face_2d, face_3d, nose_2d, nose_3d
    if idx == 1:
    # Dapatkan koordinat hidung dalam 2D dan 3D
        nose_2d = (lm.x * img_w, lm.y * img_h)
        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
    
    x, y = int(lm.x * img_w), int(lm.y * img_h)
    
    # Dapatkan koordinat 2D
    face_2d.append([x, y])
    # Dapatkan koordinat 3D
    face_3d.append([x, y, lm.z])
    
    

In [5]:
nose_tip = [1]
right_eye = [33]
left_eye = [263]
lips_upper = [61, 291]
chin = [199]

chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [ 38, 76, 268, 292, 86, 316 ]
nose_idxs = [1]
forehead_idxs = [8]

In [6]:
''' 
    Baca Kamera
'''
cap = cv2.VideoCapture(0)
looking_down_timer = None
# Loop utama untuk memproses setiap frame dari kamera
while cap.isOpened():
    start = time.time()
    # Baca frame terbaru dari kamera
    success, image = cap.read()
    # Balikkan gambar secara horizontal dan konversi ke RGB
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    # Optimalkan kinerja dengan mengubah flag writeable gambar
    image.flags.writeable = False
    # Proses deteksi wajah dan landmark dengan MediaPipe
    results = face_mesh.process(image)
    # Proses deteksi wajah dan landmark dengan MediaPipe
    image.flags.writeable = True
    # Konversi gambar kembali ke format BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    
    
    # Cek apakah terdapat wajah yang terdeteksi
    if results.multi_face_landmarks:
        face_2d = []
        face_3d = []
        nose_2d, nose_3d = [None, None]
        required_looking_time = 2
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
               praHeadPose(image, idx, lm)
            
            p1, p2, text, x, y, z = headPose(image, face_landmarks, face_2d, face_3d, nose_2d, nose_3d, required_looking_time)
            
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            # Tambahkan text pada gambar
            putText(image, text, 20, 50)
            putText(image, x, 450, 50, "x")
            putText(image, y, 450, 100, "y")
            putText(image, z, 450, 150, "z")
            
            
            # Draw landmarks on the frame
            our_landmarks = getOurLandmark()
            filtered_landmarks = [face_landmarks.landmark[idx] for idx in our_landmarks]
            for landmark in filtered_landmarks:
                x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

            
            cv2.imshow('Head Pose Estimation', image)
            
            if cv2.waitKey(1) == ord('q'):
                break

cv2.destroyAllWindows()            
cap.release()

2023-08-24 14:14:07.804715 	 0.0
2023-08-24 14:14:07.804715 	 0.032031
2023-08-24 14:14:07.804715 	 0.078611
2023-08-24 14:14:07.804715 	 0.110603
2023-08-24 14:14:07.804715 	 0.141175
2023-08-24 14:14:07.804715 	 0.171167
2023-08-24 14:14:07.804715 	 0.204171
2023-08-24 14:14:07.804715 	 0.239379
2023-08-24 14:14:07.804715 	 0.286391
2023-08-24 14:14:07.804715 	 0.316382
2023-08-24 14:14:07.804715 	 0.347952
2023-08-24 14:14:07.804715 	 0.39215
2023-08-24 14:14:07.804715 	 0.423195
2023-08-24 14:14:07.804715 	 0.454462
2023-08-24 14:14:09.113411 	 0.0
2023-08-24 14:14:09.113411 	 0.032553
2023-08-24 14:14:09.237994 	 0.0
2023-08-24 14:14:09.237994 	 0.027514
2023-08-24 14:14:09.237994 	 0.061513
2023-08-24 14:14:09.237994 	 0.092546
2023-08-24 14:14:09.237994 	 0.122249
2023-08-24 14:14:09.237994 	 0.170197
2023-08-24 14:14:09.237994 	 0.202026
2023-08-24 14:14:09.237994 	 0.233143
2023-08-24 14:14:09.237994 	 0.263151
2023-08-24 14:14:09.237994 	 0.297155
2023-08-24 14:14:09.237994 	

2023-08-24 14:14:55.954426 	 0.0
2023-08-24 14:14:55.954426 	 0.031077
2023-08-24 14:14:55.954426 	 0.064082
2023-08-24 14:14:57.765379 	 0.0
2023-08-24 14:14:57.765379 	 0.04701
2023-08-24 14:14:57.765379 	 0.078178


In [ ]:
cv2.destroyAllWindows()            
cap.release()